In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,27327
2,Huelva,Confirmados PDIA 14 días,1368
3,Huelva,Tasa PDIA 14 días,"266,57832687023796"
4,Huelva,Confirmados PDIA 7 días,637
5,Huelva,Total Confirmados,27501
6,Huelva,Curados,22877
7,Huelva,Fallecidos,359


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  27327.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7109.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 992 personas en los últimos 7 días 

Un positivo PCR cada 458 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,27327.0,637.0,1368.0,513170.0,124.130405,266.578327,120.0
Condado-Campiña,8330.0,317.0,671.0,156231.0,202.904673,429.492226,54.0
Huelva-Costa,16146.0,265.0,582.0,289548.0,91.521958,201.002943,51.0
Huelva (capital),7109.0,145.0,314.0,143837.0,100.808554,218.302662,35.0
Sierra de Huelva-Andévalo Central,2436.0,47.0,91.0,67391.0,69.742250,135.032868,13.0
Moguer,1011.0,43.0,102.0,21867.0,196.643344,466.456304,11.0
Almonte,1314.0,127.0,232.0,24507.0,518.219284,946.668299,10.0
Rociana del Condado,467.0,36.0,79.0,7939.0,453.457614,995.087543,10.0
Cartaya,1203.0,78.0,168.0,20083.0,388.388189,836.528407,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Rociana del Condado,467.0,36.0,79.0,7939.0,453.457614,995.087543,10.0
Almonte,1314.0,127.0,232.0,24507.0,518.219284,946.668299,10.0
Cartaya,1203.0,78.0,168.0,20083.0,388.388189,836.528407,7.0
San Bartolomé de la Torre,277.0,9.0,28.0,3761.0,239.298059,744.482850,2.0
Lucena del Puerto,187.0,6.0,23.0,3261.0,183.992640,705.305121,2.0
Almonaster la Real,62.0,7.0,9.0,1788.0,391.498881,503.355705,5.0
Palos de la Frontera,556.0,30.0,57.0,11742.0,255.493102,485.436893,6.0
Moguer,1011.0,43.0,102.0,21867.0,196.643344,466.456304,11.0
Condado-Campiña,8330.0,317.0,671.0,156231.0,202.904673,429.492226,54.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Alosno,161.0,1.0,10.0,3933.0,25.425884,254.258835,0.0,0.100000
San Juan del Puerto,411.0,5.0,25.0,9411.0,53.129317,265.646584,2.0,0.200000
Bonares,225.0,6.0,25.0,6060.0,99.009901,412.541254,1.0,0.240000
Lucena del Puerto,187.0,6.0,23.0,3261.0,183.992640,705.305121,2.0,0.260870
San Bartolomé de la Torre,277.0,9.0,28.0,3761.0,239.298059,744.482850,2.0,0.321429
Aljaraque,975.0,11.0,34.0,21474.0,51.224737,158.331005,2.0,0.323529
Aracena,498.0,11.0,32.0,8255.0,133.252574,387.643852,4.0,0.343750
Villarrasa,142.0,3.0,8.0,2211.0,135.685210,361.827227,1.0,0.375000
Niebla,187.0,4.0,10.0,4116.0,97.181730,242.954325,0.0,0.400000
